<a href="https://colab.research.google.com/github/arnabmy/sparknlp/blob/main/NLP_on_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.5.1

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)
     |████████████████████████████████| 215.7MB 68kB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=836d4a7f5d53b27ae265554f361d1ad0434c6deb1c3e3ea3100e8f1ed1c21f09
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 122kB 4.4MB/s 


In [2]:
! mkdir -p data

In [3]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/20newsgroups-mld/mini_newsgroups.tar.gz

--2021-04-16 12:33:15--  https://archive.ics.uci.edu/ml/machine-learning-databases/20newsgroups-mld/mini_newsgroups.tar.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1860687 (1.8M) [application/x-httpd-php]
Saving to: ‘mini_newsgroups.tar.gz’

mini_newsgroups.tar 100%[===================>]   1.77M  3.96MB/s    in 0.4s    

2021-04-16 12:33:16 (3.96 MB/s) - ‘mini_newsgroups.tar.gz’ saved [1860687/1860687]



In [4]:
! tar xzf mini_newsgroups.tar.gz -C ./data/

In [5]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2021-04-16 12:33:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2021-04-16 12:33:17 (144 MB/s) - ‘iris.data’ saved [4551/4551]



In [6]:
! mv iris.data ./data

In [7]:
! wget https://github.com/alexander-n-thomas/spark-nlp-book-prod/raw/master/en_lemmas.txt

--2021-04-16 12:33:35--  https://github.com/alexander-n-thomas/spark-nlp-book-prod/raw/master/en_lemmas.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/alexander-n-thomas/spark-nlp-book/raw/master/en_lemmas.txt [following]
--2021-04-16 12:33:35--  https://github.com/alexander-n-thomas/spark-nlp-book/raw/master/en_lemmas.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexander-n-thomas/spark-nlp-book/master/en_lemmas.txt [following]
--2021-04-16 12:33:36--  https://raw.githubusercontent.com/alexander-n-thomas/spark-nlp-book/master/en_lemmas.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.

In [8]:
from operator import concat, itemgetter, methodcaller
import os
from time import sleep

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as fun
from pyspark.sql.types import *

In [9]:
packages = ','.join([
    "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5",
])

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("spark-nlp-book-p1c3") \
    .config("spark.jars.packages", packages)\
    .getOrCreate()

In [10]:
spark

In [11]:
def has_moon(text):
    if 'moon' in text:
        sleep(1)
        return True
    else:
        return False

In [13]:
# RDD containing filepath-text pairs
path = os.path.join('data', 'mini_newsgroups', 'sci.space')
text_pairs = spark.sparkContext.wholeTextFiles(path)

In [14]:
text_pairs

org.apache.spark.api.java.JavaPairRDD@6af55935

In [17]:
texts = text_pairs.map(itemgetter(1))
lower_cased = texts.map(methodcaller('lower'))
moon_texts = texts.filter(has_moon).persist()

In [18]:
print('This appears quickly because the previous operations are '
      'all lazy')

This appears quickly because the previous operations are all lazy


In [19]:
print(moon_texts.count())

11
